In [1]:
import torch
import torchvision
from torchvision import datasets 
import torchvision.transforms  as T
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn 
import torch.nn.functional as F


In [2]:
mnist = datasets.MNIST (
    
    root='./DATASET/',
    train=True, 
    transform=T.ToTensor() 
    
)

In [4]:
train , val , test = torch.utils.data.random_split( mnist , [0.8 , 0.1 , 0.1] )

train_loader = DataLoader(
    
    dataset= train,
    shuffle = True,
    batch_size=64
)
val_loader = DataLoader(
    
    dataset= val,
    shuffle = True,
    batch_size=64
)


In [2]:
def CNNlayer_K3P1(canal_in , canal_out , stride):
    return nn.Conv2d(canal_in , canal_out , stride=stride , kernel_size=3 , padding=1)

In [ ]:
class CNN_resnet(nn.Module):
    def __init__(self , in_channel , out_channel , stride=1 , change_size = True):
        super().__init__() 
        ## UN bloque de resnet sin la entrada residual : 
        self.conv1= CNNlayer_K3P1(in_channel , out_channel , stride )
        self.bn1= nn.BatchNorm2d(out_channel)
        self.conv2 = CNNlayer_K3P1(out_channel , out_channel , 1 )
        self.bn2= nn.BatchNorm2d(out_channel)
        # Para cambiar el map size : 
        if change_size : 
            self.residual = nn.Sequential(
                
                nn.Conv2d(in_channel ,
                                    out_channel,
                                    kernel_size=1 ,
                                    stride=stride),
                nn.BatchNorm2d(out_channel),nn
                
            )
        
        